## Task 1 (10 Points)

Select padding sizes:

In [ ]:
import torch

N = 4
C = 3
C_out = 10
H = 8
W = 16

x = torch.ones((N, C, H, W))

# torch.Size([4, 10, 8, 16])
out1 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 3), padding=1)(x)
# print(out1.shape) # for self-test

# torch.Size([4, 10, 8, 16])
out2 = torch.nn.Conv2d(C, C_out, kernel_size=(5, 5), padding=2)(x)
# print(out2.shape) # for self-test

# torch.Size([4, 10, 8, 16])
out3 = torch.nn.Conv2d(C, C_out, kernel_size=(7, 7), padding=3)(x)
# print(out3.shape) # for self-test

# torch.Size([4, 10, 8, 16])
out4 = torch.nn.Conv2d(C, C_out, kernel_size=(9, 9), padding=4)(x)
#print(out4.shape) # for self-test

# torch.Size([4, 10, 8, 16])
out5 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 5), padding=(1, 2))(x)
#print(out5.shape) # for self-test

# torch.Size([4, 10, 22, 30])
out6 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 3), padding=1)(x)
#print(out6.shape) # for self-test

# torch.Size([4, 10, 7, 15])
out7 = torch.nn.Conv2d(C, C_out, kernel_size=(4, 4), padding=1)(x)
#print(out7.shape) # for self-test

# torch.Size([4, 10, 9, 17])
out8 = torch.nn.Conv2d(C, C_out, kernel_size=(2, 2), padding=0)(x)
#print(out8.shape) # for self-test

torch.Size([4, 10, 8, 16])


## Task 2 (40 Points)

Develop an architecture according to the data from the article.
To test the functionality, test your architecture on any suitable data set.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*y87bqk95D-IndWdHM_K9-g.png)
![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*XQGAKZb8kjoF_1lSXeIQxg.png)

## Step 0. Data preparation.

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yevgeniyleonov","key":"b9d7e9d003cd28f916d25b07abc141d3"}'}

In [3]:
import os
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d jessicali9530/stanford-cars-dataset

import zipfile
zip_ref = zipfile.ZipFile('stanford-cars-dataset.zip', 'r')
zip_ref.extractall('/cars')
zip_ref.close()

print(os.listdir('/cars'))

mv: cannot stat './kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
['cars_annos.mat', 'cars_test', 'cars_train']


In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

def load_data(batch_size=32):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((258, 258)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load datasets using ImageFolder
    train_dataset = datasets.ImageFolder(root='/cars/cars_train/', transform=transform)
    test_dataset = datasets.ImageFolder(root='/cars/cars_test/', transform=transform)

    # Split the training dataset into training and validation sets
    train_len = int(0.8 * len(train_dataset))
    val_len = len(train_dataset) - train_len

    train_data, val_data = random_split(train_dataset, [train_len, val_len])

    # Create DataLoaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Usage
train_loader, val_loader, test_loader = load_data(batch_size=32)


## Step 1. Neural network architecture

In [13]:
import torch
import torch.nn as nn

class FireBlock(nn.Module):
    def __init__(self, in_channels, squeeze_channels, expand1x1_channels, expand3x3_channels):
        super(FireBlock, self).__init__()
        self.squeeze = nn.Conv2d(in_channels, squeeze_channels, kernel_size=1)
        self.expand1x1 = nn.Conv2d(squeeze_channels, expand1x1_channels, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_channels, expand3x3_channels, kernel_size=3, padding=1)

    def forward(self, x):
        squeezed = torch.relu(self.squeeze(x))
        expanded1x1 = torch.relu(self.expand1x1(squeezed))
        expanded3x3 = torch.relu(self.expand3x3(squeezed))
        return torch.cat([expanded1x1, expanded3x3], 1)

class SqueezeNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SqueezeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=7, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            FireBlock(96, 16, 64, 64),
            FireBlock(128, 16, 64, 64),
            FireBlock(128, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            FireBlock(256, 32, 128, 128),
            FireBlock(256, 48, 192, 192),
            FireBlock(384, 48, 192, 192),
            FireBlock(384, 64, 256, 256),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            FireBlock(512, 64, 256, 256),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Conv2d(512, num_classes, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)

squeezenet = SqueezeNet()


## Step 2.  Loss Function

In [14]:
loss = nn.CrossEntropyLoss()

## Step 3. Optimizer

In [16]:
optim = torch.optim.Adam(squeezenet.parameters(), lr=0.0001)

## Step 4. Train Loop

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch_size, (data, targets) in enumerate(train_loader):
        optim.zero_grad()
        outputs = squeezenet(data)
        batch_loss = loss(outputs, targets)
        batch_loss.backward()
        optim.step()

        total_loss += batch_loss.item()
        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        if batch_size % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_size + 1}/{len(train_loader)}], Loss: {batch_loss.item():.4f}')

    train_accuracy = correct_predictions / total_samples
    average_train_loss = total_loss / len(train_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')



Epoch [1/10], Step [1/204], Loss: 6.8984
